In [1]:
from wilds_dataset import WildsDataset, get_datasets

import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.transforms as T

from tqdm import tqdm

from torchvision.models.resnet import resnet34, ResNet34_Weights

import numpy as np

/home/yasin/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
torch.manual_seed(42)

In [3]:
train_dataset, test_dataset = get_datasets("../data/camelyon17_v1.0", domains=[0,4], split_rat=[90_000, 10_000])

In [4]:
dataloader = DataLoader(train_dataset, batch_size=256)

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
model = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [7]:
model.train()

i = 0
losses = []
for imgs, targets in tqdm(dataloader):
    x0, x4 = imgs
    t0, t4 = targets

    x0, x4, t0, t4 = x0.to(device), x4.to(device), t0.to(device), t4.to(device)

    optimizer.zero_grad(set_to_none=True)

    y_ = model(x0)
    loss = criterion(y_, t0)

    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if i % 20 == 0:
        print(f"Avg. train-loss: {np.array(losses).mean()}")
        losses = []
        
    i += 1

  0%|          | 1/352 [00:00<05:22,  1.09it/s]

Avg. train-loss: 0.5719367265701294


  6%|▌         | 21/352 [00:09<02:21,  2.33it/s]

Avg. train-loss: 0.17410273812711238


 12%|█▏        | 41/352 [00:18<02:15,  2.30it/s]

...

 91%|█████████ | 321/352 [02:17<00:12,  2.40it/s]

Avg. train-loss: 0.049873650819063184


 97%|█████████▋| 341/352 [02:26<00:04,  2.38it/s]

Avg. train-loss: 0.04334113737568259


100%|██████████| 352/352 [02:30<00:00,  2.33it/s]


In [8]:
val_loader = DataLoader(test_dataset, batch_size=256)

In [9]:
num_samples_val = len(val_loader.dataset)
num_samples_val

10000

In [10]:
model.eval()

with torch.no_grad():
    correct = 0
    
    for imgs, targets in tqdm(val_loader):
        x0, x4 = imgs
        t0, t4 = targets

        x0, x4, t0, t4 = x0.to(device), x4.to(device), t0.to(device), t4.to(device)

        y_ = model(x4)
        correct += (t4 == y_.argmax(axis=1)).sum().item()
        
    print(f"Eval. accuracy: {correct/num_samples_val}")

100%|██████████| 40/40 [00:12<00:00,  3.23it/s]

Eval. accuracy: 0.7916


In [11]:
model.eval()

with torch.no_grad():
    correct = 0
    
    for imgs, targets in tqdm(val_loader):
        x0, x4 = imgs
        t0, t4 = targets

        x0, x4, t0, t4 = x0.to(device), x4.to(device), t0.to(device), t4.to(device)

        y_ = model(x0)
        correct += (t0 == y_.argmax(axis=1)).sum().item()
        
    print(f"Eval. accuracy: {correct/num_samples_val}")

100%|██████████| 40/40 [00:12<00:00,  3.23it/s]

Eval. accuracy: 0.9703


## Predict domain

In [12]:
# model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
# model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [13]:
dataloader = DataLoader(train_dataset, batch_size=128)

In [15]:
model.train()

i = 0
losses = []
for imgs, targets in tqdm(dataloader):
    x0, x4 = imgs
    t0, t4 = targets

    x0, x4, t0, t4 = x0.to(device), x4.to(device), t0.to(device), t4.to(device)

    x = torch.cat([x0, x4], dim=0)
    t = torch.cat([torch.zeros(len(t0)), torch.ones(len(t4))], dim=0).to(device).to(torch.long)

    optimizer.zero_grad(set_to_none=True)

    y_ = model(x)
    loss = criterion(y_, t)

    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if i % 20 == 0:
        print(f"Avg. train-loss: {np.array(losses).mean()}")
        losses = []
        
    i += 1

  0%|          | 1/704 [00:00<03:40,  3.19it/s]

Avg. train-loss: 0.106393001973629


  3%|▎         | 21/704 [00:06<03:19,  3.42it/s]

Avg. train-loss: 0.1115318713709712


  6%|▌         | 41/704 [00:12<03:16,  3.37it/s]

...

 97%|█████████▋| 681/704 [03:19<00:06,  3.43it/s]

Avg. train-loss: 0.09119792021811009


100%|█████████▉| 701/704 [03:25<00:00,  3.41it/s]

Avg. train-loss: 0.08390408456325531


100%|██████████| 704/704 [03:25<00:00,  3.42it/s]


In [17]:
val_loader = DataLoader(test_dataset, batch_size=128)

num_samples_val = len(val_loader.dataset) * 2
num_samples_val

20000

In [19]:
model.eval()

with torch.no_grad():
    correct = 0
    
    for imgs, targets in tqdm(val_loader):
        x0, x4 = imgs
        t0, t4 = targets

        x0, x4, t0, t4 = x0.to(device), x4.to(device), t0.to(device), t4.to(device)

        x = torch.cat([x0, x4], dim=0)
        t = torch.cat([torch.zeros(len(t0)), torch.ones(len(t4))], dim=0).to(device).to(torch.long)

        y_ = model(x)
        correct += (t == y_.argmax(axis=1)).sum().item()
        
    print(f"Eval. accuracy: {correct/num_samples_val}")

100%|██████████| 79/79 [00:14<00:00,  5.45it/s]

Eval. accuracy: 0.8016
